In [2]:
import paddle
import paddle.vision.transforms as T
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddle.io import Dataset

# 设置matplotlib参数
plt.rcParams['font.sans-serif'] = ['SimHei']

class ORLDataset(Dataset):
    def __init__(self, data_path, k, train=True):
        super(ORLDataset, self).__init__()
        self.data_path = data_path
        self.k = k
        self.train = train
        self.images, self.labels = self.load_orl()

    # 图片矢量化
    def img2vector(self, image):
        img = cv2.imread(image, 0)  # 读取图片
        imgVector = paddle.reshape(paddle.to_tensor(img, dtype='float32'), [1, -1]) # 重塑为1行多列
        return imgVector

    # 读入人脸库，每个人选择k张作为训练样本，剩下的作为测试样本
    def load_orl(self):
        '''
        对训练数据集进行数组初始化，用0填充，每张图片尺寸都定为112*92,
        现在共有40个人，每个人都选择k张，则整个训练集大小为40*k,112*92
        '''
        train_images = []
        train_labels = []
        test_images = []
        test_labels = []
        sample = np.random.permutation(10) + 1 # 生成随机序列

        for i in range(40): # 共有40个人
            people_num = i + 1
            for j in range(10): # 每人10张照片
                image_path = os.path.join(self.data_path, 's' + str(people_num), str(sample[j]) + '.jpg')
                img = self.img2vector(image_path) # 读取图片并进行矢量化
                if j < self.k: # 构成训练集
                    train_images.append(img)
                    train_labels.append(people_num)
                else: # 构成测试集
                    test_images.append(img)
                    test_labels.append(people_num)
        if self.train:
            return paddle.concat(train_images, axis=0), paddle.to_tensor(train_labels, dtype='int64')
        else:
            return paddle.concat(test_images, axis=0), paddle.to_tensor(test_labels, dtype='int64')

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

    def __len__(self):
        return self.images.shape[0]

def PCA(data, r): # 降低到r维
    data = paddle.cast(data, 'float32')
    rows, cols = data.shape
    data_mean = paddle.mean(data, axis=0)
    A = data - paddle.tile(data_mean, repeat_times=[rows, 1])
    C = paddle.matmul(A, A, transpose_y=True)  # 协方差矩阵
    eig_vals, eig_vects = paddle.linalg.eigh(C)  # 特征值和特征向量
    eig_vects = paddle.matmul(A.T, eig_vects[:, :r])
    for i in range(r):
        eig_vects[:, i] = eig_vects[:, i] / paddle.norm(eig_vects[:, i])
    final_data = paddle.matmul(A, eig_vects)
    return final_data, data_mean, eig_vects

def face_recognize(data_path):
    for r in range(10, 41, 10):
        print(f"当降维到{r}时:")
        dataset_train = ORLDataset(data_path, k=7, train=True)
        dataset_test = ORLDataset(data_path, k=7, train=False)
        
        train_data, train_labels = paddle.to_tensor(dataset_train.data), paddle.to_tensor(dataset_train.labels, dtype='int64')
        test_data, test_labels = paddle.to_tensor(dataset_test.data), paddle.to_tensor(dataset_test.labels, dtype='int64')

        data_train_new, data_mean, V_r = PCA(train_data, r)
        temp_face = test_data - data_mean
        data_test_new = paddle.matmul(temp_face, V_r)

        true_num = 0
        for i in range(len(dataset_test)):
            diffMat = data_train_new - data_test_new[i]
            sqDiffMat = paddle.square(diffMat)
            sqDistances = paddle.sum(sqDiffMat, axis=1)
            sortedDistIndices = paddle.argsort(sqDistances)
            if train_labels[sortedDistIndices[0]] == test_labels[i]:
                true_num += 1

        accuracy = float(true_num) / len(dataset_test)
        print(f'当每个人选择7张照片进行训练时，The classify accuracy is: {accuracy:.2%}')


if __name__ == '__main__':
    # 以下部分用于演示如何调用PCA函数和ORLDataset
    # 注意：需要先设置正确的orlpath路径
    orlpath = "D:\\Users\\Yu\\Desktop\\Paddle\\PCA\\ORL"  # 请将此路径替换为实际路径
    face_recognize(orlpath)
